# 要約 
このJupyter Notebookは、Kaggleコンペティション「LLM 20 Questions」におけるエージェント作成プロセスを示しています。具体的には、20の質問ゲームを効果的にプレイするための言語モデル（LLM）エージェントを生成し、最終的に `submission.tar.gz` ファイルを作成します。このファイルはコンペティションに提出するために使用されます。

### 問題と目的
「LLM 20 Questions」は、プレイヤーが質問を通じて特定のターゲットを推測するゲームです。このノートブックは、質問者と回答者の役割を持つAIエージェントを開発し、効率的にターゲットを推測する能力を高めることを目的としています。

### 使用されている手法とライブラリ
- **ライブラリと依存関係**: 
  - `immutabledict` や `sentencepiece` などのライブラリをインストールし、モデルの動作に必要なファイルを適切なディレクトリに配置します。
  - GitHubから`gemma_pytorch`リポジトリをクローンし、モデル用のファイルを作成した作業ディレクトリに移動させます。

- **モデルの設定と初期化**: 
  - モデルの初期化にはGemmaライブラリを使用し、特に`GemmaForCausalLM`クラスが利用されます。ここでは7Bおよび2Bのモデル設定が用意されており、指定したバリアントに応じて適切なモデルを動的に選択しています。

- **プロンプトフォーマッタ**: 
  - `GemmaFormatter`クラスを定義して、ゲームのプロンプトを適切にフォーマットし、ユーザーのインプットをモデルに渡すための準備を行います。

- **エージェントクラス**: 
  - `GemmaQuestionerAgent`と`GemmaAnswererAgent`という2つのエージェントクラスが実装され、質問者と回答者の役割をそれぞれ担当します。それぞれのクラスは内部でプロンプトを生成し、モデルからの応答を解析して正しい質問や答えを生成します。

### 出力
最終的に、ノートブックは`submission.tar.gz`ファイルを生成し、それをコンペティションに提出できる状態にします。この成果物には、すべての必要なモデルファイルと実行可能なコードが含まれています。

このノートブックは、言語モデルが協調プレイに基づいた推理を行うための出発点として機能します。参加者はこの基盤を元に、自らのモデルや戦略を実装し、さらに改善することが期待されています。

---


# 用語概説 
以下は、ノートブックに関連した専門用語の簡単な解説です。初心者がつまずきそうなマイナーな概念や特有のドメイン知識に焦点を当てています。

1. **LLM (Large Language Model)**: 大規模なデータセットを使って訓練された言語モデルであり、人間のように文章を理解し生成する能力を持つ。特定のタスク（この場合は20の質問ゲーム）に調整されることが多い。

2. **Few-shot examples**: モデルに対して数例の入力を与え、その例を基にモデルがタスクを理解して実行できるようにする。例えば、質問と答えのペアを示すことで、モデルがその形式を学ぶ。

3. **Gemma**: Googleが開発した特定のLLMアーキテクチャで、テキスト生成タスクのためにチューニングされている。gemma_pytorchはそのPyTorch実装を含むリポジトリ。

4. **Causal LM (Causal Language Model)**: 文脈から次の単語を予測するために自動回帰モデルを使用した言語モデル。過去の情報を基に未来の情報を生成する能力がある。

5. **Tokenization**: 文を構成する単語や記号を「トークン」と呼ばれる小さな単位に分割するプロセス。トークン化は、モデルがテキストを理解しやすくするための重要なステップです。

6. **Weight files (重みファイル)**: ニューラルネットワークにおける学習結果（モデルがパターンを学ぶ過程で得られるパラメータ）のこと。これらのファイルは、訓練されたモデルを再利用するために必要です。

7. **Sampling parameters**: 生成するテキストの多様性や一貫性を調整するための設定（例：temperature, top_k, top_p）。これらのパラメータを調整することで、モデルの応答のスタイルや創造性が変わる。

8. **Device (デバイス)**: モデルを実行するためのハードウェア。多くの場合、GPU（グラフィックス処理ユニット）を意味します。深層学習のトレーニングや推論においては、GPUを利用することで計算速度が大幅に向上します。

9. **Interleave**: 異なる2つのリストの要素を交互に組み合わせる操作。ここでは、質問と回答を交互に結合してゲームの進行状況を保持します。

10. **Context manager**: Pythonの機能で、特定の処理が始まる前に状態を設定し、処理が終わった後に元の状態に戻すための構文。リソース管理に便利です。

これらの用語は、このノートブックだけでなく、機械学習や深層学習における実務や研究でもよく使われる概念です。初心者はこれらを理解することで、学習を深めることができるでしょう。

---


このノートブックは、**LLM 20 Questions** のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz` ファイルが生成されます。このファイルは、右側の **Submit to competition** 見出しから直接提出することができます。あるいは、ノートブックビューアから *Output* タブをクリックし、`submission.tar.gz` を見つけてダウンロードします。競技のホームページの左上にある **Submit Agent** をクリックしてファイルをアップロードし、提出を行ってください。



In [ ]:
%%bash
cd /kaggle/working  # 作業ディレクトリに移動します
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece  # 必要なライブラリをインストールします
git clone https://github.com/google/gemma_pytorch.git > /dev/null  # gemma_pytorchリポジトリをクローンします
mkdir /kaggle/working/submission/lib/gemma/  # gemma用のディレクトリを作成します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/  # gemmaのファイルを移動させます

In [ ]:
%%writefile submission/main.py
# 設定
import os
import sys

# **重要:** コードがノートブックとシミュレーション環境の両方で動作するように、システムパスを次のように設定します。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):  # KAGGLE_AGENT_PATHが存在するか確認します
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))  # KAGGLE_AGENT_PATHのlibディレクトリをパスに追加します
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")  # 指定のパスが存在しない場合、別のパスを追加します

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b  # gemmaモデル設定の取得
from gemma.model import GemmaForCausalLM  # gemmaモデルのインポート

if os.path.exists(KAGGLE_AGENT_PATH):  # KAGGLE_AGENT_PATHが存在する場合
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")  # 重みファイルのパスを設定
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"  # 他のパスを設定

# プロンプトフォーマット
import itertools
from typing import Iterable

class GemmaFormatter:
    _start_token = '<start_of_turn>'  # ターンの開始トークン
    _end_token = '<end_of_turn>'  # ターンの終了トークン

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt  # システムプロンプトの保存
        self._few_shot_examples = few_shot_examples  # Few-shot例の保存
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"  # ユーザーのターンをフォーマット
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"  # モデルのターンをフォーマット
        self.reset()  # 状態をリセット

    def __repr__(self):
        return self._state  # 現在の状態を返す

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)  # ユーザーのプロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)  # モデルのプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"  # ユーザーターンの開始を追加
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"  # モデルターンの開始を追加
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"  # ターンの終了を追加
        return self

    def reset(self):
        self._state = ""  # 状態を初期化
        if self._system_prompt is not None:
            self.user(self._system_prompt)  # システムプロンプトを追加
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # Few-shot例のターンを適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]  # ターンの実行順序を決定
        formatters = itertools.cycle(formatters)  # 順序を循環させる
        for fmt, turn in zip(formatters, turns):  # フォーマッタとターンを結合
            fmt(turn)  # フォーマッタでターンを処理
        return self


# エージェント定義
import re

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたdtypeにデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)  # デフォルトのdtypeを変更
    yield
    torch.set_default_dtype(torch.float)  # 元のdtypeに戻す

class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant  # エージェントのバリアントを保存
        self._device = torch.device(device)  # 使用するデバイスを指定
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)  # フォーマッタを初期化

        print("モデルの初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()  # モデル設定を取得
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # トークナイザーのパスを設定
        model_config.quant = "quant" in variant  # 量子化設定

        with _set_default_tensor_type(model_config.get_dtype()):  # データ型を設定しモデルを初期化
            model = GemmaForCausalLM(model_config)  # モデルを初期化
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')  # 重みのチェックポイントパスを設定
            model.load_weights(ckpt_path)  # 重みをロード
            self.model = model.to(self._device).eval()  # モデルを指定デバイスに移動し評価モードにする

    def __call__(self, obs, *args):
        self._start_session(obs)  # セッションを開始
        prompt = str(self.formatter)  # フォーマッタの内容を文字列に変換
        response = self._call_llm(prompt)  # LLMにプロンプトを渡して応答を得る
        response = self._parse_response(response, obs)  # 応答を解析
        print(f"{response=}")  # 応答をコンソールに出力
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError  # メソッドを抽象化

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {  # サンプリングのパラメータを設定
                'temperature': 0.01,  # 温度パラメータ
                'top_p': 0.1,  # トップ確率
                'top_k': 1,  # トップKの設定
        }
        response = self.model.generate(  # モデルから新しいトークンを生成
            prompt,
            device=self._device,
            output_len=max_new_tokens,  # 最大トークン数
            **sampler_kwargs,  # その他のサンプリングパラメータ
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # キーワードを抽出
        if match is None:
            keyword = ''  # キーワードが見つからない場合
        else:
            keyword = match.group().lower()  # キーワードを小文字に変換
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError  # メソッドを抽象化


def interleave_unequal(x, y):
    return [  # 不均一なリストを交互に結合
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")  # ゲーム開始メッセージ
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に結合
        self.formatter.apply_turns(turns, start_agent='model')  # ターンを適用
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")  # 質問するよう指示
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")  # キーワード予想の指示
        self.formatter.start_model_turn()  # モデルターンを開始

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))  # 質問を抽出
            if match is None:
                question = "Is it a person?"  # デフォルトの質問
            else:
                question = match.group()  # 抽出した質問
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)  # キーワードの予想を解析
            return guess
        else:
            raise ValueError("Unknown turn type:", obs.turnType)  # エラーハンドリング


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")  # ゲーム開始メッセージ
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に結合
        self.formatter.apply_turns(turns, start_agent='user')  # ターンを適用
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")  # 回答指示メッセージ
        self.formatter.start_model_turn()  # モデルターンを開始

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)  # 回答を解析
        return 'yes' if 'yes' in answer else 'no'  # "yes"であればyes、その他はnoを返す


# エージェント作成
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."  # システムプロンプトの設定

few_shot_examples = [  # Few-shot例の設定
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is it a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]

# **重要:** エージェントはグローバルに定義します。必要なエージェントだけをロードするため。
# 両方をロードすると、OOM（Out of Memory）に繋がる可能性があります。
agent = None

def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )  # 質問者エージェントの初期化
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )  # 回答者エージェントの初期化
    assert agent is not None, "Agent not initialized."  # エージェントが初期化されているか確認

    return agent  # エージェントを返す

def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)  # 質問者エージェントの応答を取得
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)  # 質問者エージェントの応答を取得
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)  # 回答者エージェントの応答を取得
    if response is None or len(response) <= 1:  # 応答が空か長さが1以下の場合
        return "yes"  # デフォルトの応答
    else:
        return response  # 通常の応答を返す

In [ ]:
!apt install pigz pv > /dev/null  # pigzとpvをインストールします

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2  # submission.tar.gzファイルを作成

---

# コメント 

> ## Samar Elhissi
> 
> 例をありがとう、ローカルでテストするにはどうすれば良いですか？
> 
> 
> 
> > ## Valentin Baltazar
> > 
> > ハードウェアがあるか確認してください…このLLMは多くの計算を必要とし、トレーニングとファインチューニングには強力なGPUが必要です。クラウドを利用するのが、はるかに簡単です。
> > 
> > 

---

> ## Michael Kamal 92
> 
> ありがとうございます、few_shot_examplesについて質問したいのですが、どのように作成すれば良いですか？
> > 例えば、( 'is it place?', 'yes-or-no' )のようにする必要がありますか、それとも( 'is it place?', 'yes', ) のようにすれば良いですか？それとも( 'is it place?', 'yes', 'Now guess the keyword' )でしょうか？それとも( 'is it place?', 'no', 'Now guess the keyword', 'France' )でしょうか？それとも( 'is it place?', 'yes', 'France' )でしょうか？どれが正しい質問、回答、予測の作り方ですか？
> 
> もう一つの質問ですが、Gemmaはfew_shot_examplesでトレーニングしますか？

---

> ## Yukky_2801
> 
> こんにちは、私はKaggleの初心者です。あなたのノートブックを実行すると、以下のエラーが出ました：
> 
> tar: gemma/pytorch/7b-it-quant/2: Cannot stat: No such file or directory
> 
> 1.37MiB 0:00:00 [36.4MiB/s] [<=> ]
> 
> tar: 前のエラーのために失敗したと終了します。
> 
> submission.tar.gzをエラーと共に提出できません。どういうことかわからないのですが、解決策を提供していただけますか？

> ## Andres H. Zapke
> > もちろん「gemma/pytorch/7b-it-quant/2」このパスにアクセスしようとしています。ファイルがそのパスにあることを確認してください（ノートブックの右側を見て、gemmaモデルがそこのパスと一致しているか確認してください）。
>   
> > ## Aryan Singh
> > > Gemma 7b-it-quant V2を追加するには、Add Input機能を使用します。
> > > 
> > > まず、こちらでライセンスを受け入れることを確認してください：[https://www.kaggle.com/models/google/gemma](https://www.kaggle.com/models/google/gemma)
> > 
> > 
> > > ## Talal Mufti
> > > > すべてのファイルがそのパスにあることを確認した後、依然として問題が発生したため、bashコマンドを少し修正しました。個人的には、これが私にとってうまくいきました：
> > > > !tar --use-compress-program='pigz --fast --recursive | pv' -f submission.tar.gz -c /kaggle/working/submission . -c /kaggle/input/gemma/pytorch/7b-it-quant/2

---

> ## Muhammad Hadi13
> 
> なぜファイルをコピーして実行しているのに、常に1.35MB以上の出力が生成されず、バリデーションエピソードで失敗するのかわかりません。Ryanの出力は約7GBでした。この件について助けが必要です！！
> 
> 
> 1.37MiB 0:00:00 [36.4MiB/s] [<=> ]
> > tar: gemma/pytorch: Cannot stat: No such file or directory
> 
> tar: 前のエラーのために終了しました。

> ## Aryan Singh
> > > Gemma 7b-it-quant V2を事前に追加する必要があります。
> > > 
> > > ノートブック内でモデルを追加するには、Add input機能を使用します。
> > > 
> > > まず、こちらでライセンスを受け入れることを確認してください：[https://www.kaggle.com/models/google/gemma](https://www.kaggle.com/models/google/gemma)

---

> ## Ship of Theseus
> 
> Thank Ryan, greate work! Nice code to run on localhost and sharing to Kaggle Community
> 
> 

---

> ## shiv_314
> 
> 皆さん！一つ助けが必要です。gemmaパッケージでインポートエラーが発生しています。
> 
> Pythonのためにすでに正しいシステムパスを追加しましたが、それでも同じ問題が発生しています。助けてください！

---

> ## dedq
> 
> Thank Ryan, greate work! Nice code to run on localhost and sharing to Kaggle Community

---

> ## Code Hacker
> 
> このノートブックの出力ファイルtar.gzを提出しようとしたが、失敗しました…

> ## Code Hacker
> > > このモデルに同意しなかった。下の赤いボタンをクリック…

---

> ## JAPerez
> 
> Great work Ryan!

---

> ## philipha2
> 
> こんにちは、私はこのコンペの初心者です。あなたのノートブックを実行して提出しようとしました。
> 提出物のファイルであるsubmission.tar.gzをどこに置けばよいのでしょうか？ 
> Submit agentsボタンをクリックした後、このファイルをそのまま提出すればよいですか？ 
> 時間がかかります
> 基本的な質問かもしれませんが、返信ありがとうございます！

> ## Kanchan Maurya
> > > Submit agentsをクリックした後、このファイルを提出しています。それに時間がかかるのは、初期シミュレーションが機能しているためです。

---

> ## vj4science
> 
> Thanks Ryan - this is a good head start to the competition! much appreciated!

---

> ## gb_kwon
> 
> Thank you so much for your COOL guidelines!

---

> ## Andres H. Zapke
> 
> main.pyで、gemma_pytorchライブラリを次のようにインポートしています：from gemma.config。
> 
> これは私には機能しませんが、gemmaとインポートするとエラーは出ません。
> 
> 自分のローカルのgemmaモジュールのパスを手動で指定するのと、Pythonライブラリ名でインポートするのを試みましたが。何かアイデアはありますか？

---

> ## Duy Thai
> 
> こんにちは[@ryanholbrook](https://www.kaggle.com/ryanholbrook)、あなたのノートブックを試したところ、「添付されたモデルはアクセスするために追加の手順が必要です。詳細はModelパネルを参照してください。」というメッセージが表示されました。それについてはどうすればよいでしょうか？
> 
> パネルを開いたとき、私はこれだけを見ています：

> ## Andres H. Zapke
> > > "Models"へ行き、Gemmaを検索し、モデルのライセンスを受け入れます。

> ## Duy Thai
> > > ありがとうございます！

---

> ## Kai_Huang
> 
> こんにちは、私はKaggleの初心者です。あなたの !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2 のコードブロックを実行したとき、次のエラーが出ました：
> 
> tar: gemma/pytorch/7b-it-quant/2: Cannot stat: No such file or directory
> 
> 1.37MiB 0:00:00 [36.4MiB/s] [<=> ]
> 
> tar: 前のエラーのために終了しました
> 
> どういうことかわからないのですが、教えていただけますか？ありがとうございます！

> ## Kai_Huang
> > > ああ、わかりました。モデルをノートブックに入力していなかったのですね😱

> ## D Prince Armand KOUMI
> > > モデルがない場合は、追加してみてください。

---

> ## Qusay AL-Btoush
> 
> とても良い、Ryan 

---